### 02c_MCLP_and_p-median — Optimal facility siting on sparse travel matrices

This notebook builds on **02b_matrix_coverage_fast_ops** to **choose station locations** that best serve demand under travel-time constraints. We solve two classic models:

- **MCLP (Maximal Covering Location Problem):** place `p` stations to **maximise population covered** within a response-time threshold `T` (ARP-aligned bands: 7 / 15 / 18 / 40 min).
- **p-median:** place `p` stations to **minimise population-weighted travel time**.

Both models re-use 02b’s cached sparse matrices (**R**, **R_mask**) for speed and reproducibility, and export clean tables/layers for maps, dashboards, and scenario comparisons.

---

### What you’ll do here

- **Load 02b outputs:** `R.npz` (csr), `R_mask.npz` (csr), `matrix_metadata.npz`, plus `lsoa_index`, `population` (aligned to rows), and `station_lsoas` (aligned to columns).
- **Build coverage sets** once per threshold `T` via boolean CSR incidence `A_T = (R ≤ T)`.
- **Solve MCLP** for a grid of `(T, p)` and report coverage KPIs vs the 02b baseline.
- **Solve p-median** for a range of `p`, with optional **k-nearest** edge sparsification to keep the model lean.
- **(Optional) Variants:**  
  - **LSCP** (feasibility at `T`): minimal `p` so every LSOA is within `T`.  
  - **Back-up MCLP**: enforce ≥2-deep cover for resilience.
- **Export** selected sites, assignments, KPI deltas, and **map-ready** layers.

---

### Inputs (contracts match 02b)

- **Matrices dir (from 02b):** `R.npz`, `R_mask.npz`, `matrix_metadata.npz`
- **Demand & weights:** `lsoa_index`, `population`
- **Candidate sites:** `station_lsoas` (candidate set `J`)
- **Parameters:**  
  - Coverage thresholds: `T ∈ {7, 15, 18, 40}` (minutes)  
  - Facility counts: `p` (e.g., `{6, 8, 10, 12, 14}`)  
  - p-median sparsification: `k` (default `15`) and optional max radius (e.g., `60` min)

---

### Models (brief)

- **MCLP:**  
  Variables: `x_j ∈ {0,1}` (open j), `y_i ∈ [0,1]` (i covered)  
  Constraints: `y_i ≤ ∑_{j∈S_i(T)} x_j`, `∑_j x_j = p`  
  Objective: maximise `∑_i w_i y_i`  
  _Implemented via CSR `A_T` so `y ≤ A_T x` without Python loops._

- **p-median:**  
  Variables: `x_j ∈ {0,1}`, `z_ij ∈ [0,1]` (assignment)  
  Constraints: `∑_j z_ij = 1`, `z_ij ≤ x_j`, `∑_j x_j = p`  
  Objective: minimise `∑_i ∑_j w_i d_ij z_ij`  
  _Edge set reduced to row-wise `k` nearest candidates per demand to shrink z-vars._

---

### KPIs & artifacts

- **Coverage vs baseline (02b):** population % within `{7,15,18,40}` for each `(T, p)`
- **p-median costs:** population-weighted mean minutes (overall + deciles)
- **Assignments:** nearest chosen site per LSOA with travel time
- **Top gains:** LSOAs with largest minutes saved vs baseline
- **Map layers:** binary coverage (≤`T`) and “minutes to assigned station” rasters
- **Chosen sites CSV:** station LSOA, selection flag, (optional) opening rank

---

### Sanity checks

1. **MCLP monotonicity:** for fixed `T`, coverage must not decrease as `p` grows.  
2. **Threshold monotonicity:** for fixed `p`, coverage must not decrease as `T` grows.  
3. **LSCP feasibility:** report minimal `p` for feasibility at `T` (or minimal `T` for a given `p`).  
4. **p-median extremes:** `p = |J|` → cost equals baseline nearest-open; `p = 1` → single medoid.

---

### Notes aligned to your codebase

- Re-use 02b paths: `MATRICES_DIR`, `TABLES_DIR`, `MAPS_DIR`; keep column names consistent (`resp_le_{T}`, `t_resp_min`, etc.).
- Preserve index order for `station_lsoas` (02b emphasised index stability).
- After choosing `x`, you can pass the selected columns back into **02b**’s scenario runner to recompute end-to-end KPIs.

> **Tip:** Start with **PuLP + CBC** (lightweight). For larger scales, switch to OR-Tools MIP or python-mip. Warm-start p-median from a greedy MCLP solution or k-medoids for faster convergence.
